In [1]:
import pandas as pd
import numpy as np
import librosa
from librosa import display
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

In [7]:
# Here we will convert all the mp3 files to wav files
import subprocess
import os 

# Get a list of file paths
base_path = 'train_audio/'
#bird_paths = os.walk(base_path)
#list(os.walk(base_path))
bird_dirs = os.listdir(base_path)
bird_dirs = [base_path + bird_dir + '/' for bird_dir in bird_dirs]
bird_dirs = list(filter(lambda x: '.' not in x, bird_dirs))
bird_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
print(len(bird_paths))

wav_paths = [bird_dir + bird_file for bird_dir in bird_dirs for bird_file in os.listdir(bird_dir)]
wav_paths = list(filter(lambda x: '.wav' in x, wav_paths))
print(len(wav_paths))
for wav_path in wav_paths:
    subprocess.run(['rm', wav_path])

def convert_to_wav(path):
    wav_path = path.strip('.mp3')
    wav_path = wav_path + '.wav'
    subprocess.run(['ffmpeg', '-i', path, wav_path])
    return wav_path

def delete_file(path):
    subprocess.run(['rm', path])
    return

#convert_to_wav(bird_paths)

21375
0


In [8]:
meta_csv_path = 'train.csv'
meta_data = pd.read_csv(meta_csv_path)
# We drop all audio files with a rating below 4 points, this still leaves some 15 thousand files.
meta_data = meta_data[meta_data.rating >= 4]
meta_data.shape
meta_data.columns

Index(['rating', 'playback_used', 'ebird_code', 'channels', 'date', 'pitch',
       'duration', 'filename', 'speed', 'species', 'number_of_notes', 'title',
       'secondary_labels', 'bird_seen', 'sci_name', 'location', 'latitude',
       'sampling_rate', 'type', 'elevation', 'description', 'bitrate_of_mp3',
       'file_type', 'volume', 'background', 'xc_id', 'url', 'country',
       'author', 'primary_label', 'longitude', 'length', 'time', 'recordist',
       'license'],
      dtype='object')

In [9]:
#test_dataset = meta_data.sample(n=10)
#test_dataset.shape

def get_fp_from_df(df):
    base_path = "train_audio/"
    codes = df['ebird_code']
    filenames = df['filename']
    path_suffix = list(zip(codes,filenames))
    path_suffix = list(map(lambda x: x[0]+'/'+x[1], path_suffix))
    paths = [base_path + path for path in path_suffix]
    return paths

def get_mpccs(path):
    data, sample_rate = librosa.load(path, res_type='kaiser_fast')
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
    mfcc_processed = np.mean(mfcc.T, axis=0)
    return (mfcc_processed,sample_rate)
    
def extract_features(df):
    mp3_paths = get_fp_from_df(df)
    mfcc_list = []
    librosa_srs = []
    for path in mp3_paths:
        wav_path = convert_to_wav(path)
        extracted = get_mpccs(wav_path)
        mfcc_list.append(extracted[0])
        librosa_srs.append(extracted[1])
        delete_file(wav_path)
    df['mfccs'] = mfcc_list
    df['librosa_sample_rate'] = librosa_srs
    return df

feature_df = extract_features(meta_data)
feature_df.to_csv('feature.csv', index=False)

#test_df = pd.read_csv('feature.csv')

In [61]:
# Convert features and output labels to numpy arrays
X = np.array(feature_df.mfccs.tolist())
y = np.array(feature_df.ebird_code.tolist())
print(type(X[1057][0]))
# Label Encoder
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=127)

<class 'numpy.float32'>


In [70]:
# The actuall feedforward neural network architecture
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

model = build_model_graph()
model.layers
#model.summary()
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: {}".format(accuracy))

Pre-training accuracy: 0.2585649723187089


In [72]:
from keras.callbacks import ModelCheckpoint
#from datetime import datetime
import time

num_epochs = 100
num_batch_size = 32

start_time = time.time
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test,y_test),verbose=1)
stop_time = time.time
print("Training time: {}".format(stop_time-start_time))

Epoch 1/100
387/387 [==============================] - 1s 1ms/step - loss: 9.4453 - accuracy: 0.0048 - val_loss: 5.5488 - val_accuracy: 0.0058
Epoch 2/100
387/387 [==============================] - 1s 1ms/step - loss: 5.5596 - accuracy: 0.0053 - val_loss: 5.5250 - val_accuracy: 0.0058
Epoch 3/100
387/387 [==============================] - 1s 1ms/step - loss: 5.5206 - accuracy: 0.0059 - val_loss: 5.5072 - val_accuracy: 0.0048
Epoch 4/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4985 - accuracy: 0.0065 - val_loss: 5.4944 - val_accuracy: 0.0045
Epoch 5/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4856 - accuracy: 0.0065 - val_loss: 5.4850 - val_accuracy: 0.0036
Epoch 6/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4719 - accuracy: 0.0051 - val_loss: 5.4781 - val_accuracy: 0.0036
Epoch 7/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4637 - accuracy: 0.0063 - val_loss: 5.4730 - val_accuracy: 0.0036

Epoch 58/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4380 - accuracy: 0.0074 - val_loss: 5.4597 - val_accuracy: 0.0036
Epoch 59/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4366 - accuracy: 0.0062 - val_loss: 5.4597 - val_accuracy: 0.0036
Epoch 60/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4356 - accuracy: 0.0072 - val_loss: 5.4597 - val_accuracy: 0.0036
Epoch 61/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4360 - accuracy: 0.0070 - val_loss: 5.4597 - val_accuracy: 0.0036
Epoch 62/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4362 - accuracy: 0.0069 - val_loss: 5.4597 - val_accuracy: 0.0036
Epoch 63/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4363 - accuracy: 0.0066 - val_loss: 5.4596 - val_accuracy: 0.0036
Epoch 64/100
387/387 [==============================] - 1s 1ms/step - loss: 5.4368 - accuracy: 0.0068 - val_loss: 5.4596 - val_accuracy:

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'builtin_function_or_method'

In [76]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 0.74%
Testing Accuracy: 0.36%
